In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import statistics
import sklearn
import sklearn.metrics as sm

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score

import catboost as cb
from catboost import CatBoostClassifier
pd.options.display.max_columns = 130

In [ ]:
dataset = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")
testset = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
target = dataset['target']
dataset = dataset.drop(columns=['id','target'])

In [ ]:
dataset.isnull().sum().sort_values(ascending=False)

In [ ]:
dataset['std'] = dataset.std(axis=1)
dataset['var'] = dataset.var(axis=1)
dataset['min'] = dataset.min(axis=1)
dataset['max'] = dataset.max(axis=1)

dataset.head()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset,target, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))

In [ ]:
cat_params = {
    'iterations': 15585, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli',
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 8, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'eval_metric' : 'AUC',
    'verbose' : 1000,
    'early_stopping_rounds' : 500,
}

In [ ]:
cat = CatBoostClassifier(**cat_params )

cat.fit(X_train, Y_train)

test_predict = cat.predict(X_test)


In [ ]:
y_pred_proba = cat.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(Y_test,  y_pred_proba)
auc = metrics.roc_auc_score(Y_test, y_pred_proba)
print(auc)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
testset.isnull().sum().sort_values(ascending = False)

In [ ]:
testid = testset['id']
testset= testset.drop(columns=['id']) 

In [ ]:
testset['std'] = testset.std(axis=1)
testset['var'] = testset.var(axis=1)
testset['min'] = testset.min(axis=1)
testset['max'] = testset.max(axis=1)

In [ ]:
y_prdict =cat.predict_proba(testset)[:, 1]


In [ ]:
submission = pd.DataFrame(list(zip(testid, y_prdict)),
               columns =['id', 'target'])
submission.head(20)

In [ ]:
submission.to_csv('submission.csv', index=False)